# Data Preparation for Deep Learning

In this very first run, we will try to ignore DNS entries and try to ascertain if knowledge of the MAC address
and OUI as well as other session summary information is sufficient to get our model to converge with sufficient accuracy. I think that if we can get > 70% prediction that a session summary entry is IoT then we win.  

Also, for the first run we will only try to predict that the session info is pertaining to an IoT packet.  We will try to classify later.

In [1]:
from pathlib import Path
import os
import re
import apsw
import pandas as pd
import time
import pprint


In [2]:
cwd = Path.cwd()
db_path = os.path.join(cwd.parent, 'Data', 'NetCollector.sqlite')


The following query relies on the existence of the following view in the database

`create view if not exists v_session_mnf_devicetype as
select sessionid, manufacturer, device_type from devicelog group by sessionid`

In [3]:
conn = apsw.Connection(db_path)
cur = conn.cursor()

sql = """
-- We still have the smell on some endtime entries being less than
-- some starttime entries

select s.srcport,
       s.dstport,
       s.lensum,
       s.pktcount,
       s.endtime - s.starttime as durn, -- SMELL --
       d.manufacturer,
       d.device_type
from sessions s
         inner join v_session_mnf_devicetype d on s.sessionid = d.sessionid
where d.device_type is not null

"""


In [4]:
arr = []
for row in cur.execute(sql):
    arr.append(row)


In [5]:
cols = ['SourcePort',
        'DestnPort',
        'SessionPackets',
        'SessionPktLen',
        'SessionPktArrivalTime',
        'Manufacturer',
        'DeviceType']
df = pd.DataFrame(data=arr, columns=cols)

In [6]:
df.describe()

,SessionPackets,SessionPktLen,SessionPktArrivalTime
count,9.688000e+03,9688.000000,9688.000000
mean,2.904122e+05,349.180429,6.928621
std,1.106329e+07,8618.014923,132.783338
min,6.000000e+01,1.000000,-2039.997730
25%,2.420000e+02,2.000000,0.000000
50%,1.178000e+03,6.000000,0.758710
75%,3.627250e+03,13.000000,18.063444
max,7.402338e+08,496770.000000,2088.069581


# What we will do

The following is done next...

1. Normalise the values in place (all non-categorical is between 0 and 1)
2. Create category maps for `Manufacturer`
3. Drop features that we are not going to use in our model (i.e. `SourcePort`, `DestnPort`)
3. We will create an `is_iot` feature to try to initially train our model to distinguish only if a given record describing one session's worth of packets relates to an IOT device. This will be our `y-value` initially





In [7]:
df

,SourcePort,DestnPort,SessionPackets,SessionPktLen,SessionPktArrivalTime,Manufacturer,DeviceType
0,55630,63960,36384156,126998,-419.030715,Ubiquiti Networks Inc.,Other
1,63960,55630,9039457,125631,-419.074508,"Apple, Inc.",Other
2,49322,7550,306906857,228847,-419.074864,Ubiquiti Networks Inc.,UVC-G3-Flex Camera
3,49323,7550,84313180,83489,-419.072925,Ubiquiti Networks Inc.,UVC-G3-Flex Camera
4,443,58235,18288,127,-434.025740,Ubiquiti Networks Inc.,Other
...,...,...,...,...,...,...,...
9683,53,61777,151,1,0.000000,Ubiquiti Networks Inc.,Other
9684,55463,443,1999,5,0.203312,Technicolor CH USA Inc.,Other
9685,443,55463,1840,6,0.175634,Ubiquiti Networks Inc.,Other
9686,54459,53,87,1,0.000000,Intel Corporate,Other


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9688 entries, 0 to 9687
Data columns (total 7 columns):
SourcePort               9688 non-null object
DestnPort                9688 non-null object
SessionPackets           9688 non-null int64
SessionPktLen            9688 non-null int64
SessionPktArrivalTime    9688 non-null float64
Manufacturer             9688 non-null object
DeviceType               9688 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 529.9+ KB


In the first `1000000` session entries there are only `52` DNS packets... let's try to learn without these

In [9]:
import numpy as np
df['is_iot'] = np.where(df['DeviceType'] == 'Other', 0, 1)


In [10]:
df.head()

,SourcePort,DestnPort,SessionPackets,SessionPktLen,SessionPktArrivalTime,Manufacturer,DeviceType,is_iot
0,55630,63960,36384156,126998,-419.030715,Ubiquiti Networks Inc.,Other,0
1,63960,55630,9039457,125631,-419.074508,"Apple, Inc.",Other,0
2,49322,7550,306906857,228847,-419.074864,Ubiquiti Networks Inc.,UVC-G3-Flex Camera,1
3,49323,7550,84313180,83489,-419.072925,Ubiquiti Networks Inc.,UVC-G3-Flex Camera,1
4,443,58235,18288,127,-434.025740,Ubiquiti Networks Inc.,Other,0


In [11]:
drop_columns = ['SourcePort', 'DestnPort', 'DeviceType']

In [12]:
df.drop(drop_columns, axis=1, inplace=True)

In [13]:
df

,SessionPackets,SessionPktLen,SessionPktArrivalTime,Manufacturer,is_iot
0,36384156,126998,-419.030715,Ubiquiti Networks Inc.,0
1,9039457,125631,-419.074508,"Apple, Inc.",0
2,306906857,228847,-419.074864,Ubiquiti Networks Inc.,1
3,84313180,83489,-419.072925,Ubiquiti Networks Inc.,1
4,18288,127,-434.025740,Ubiquiti Networks Inc.,0
...,...,...,...,...,...
9683,151,1,0.000000,Ubiquiti Networks Inc.,0
9684,1999,5,0.203312,Technicolor CH USA Inc.,0
9685,1840,6,0.175634,Ubiquiti Networks Inc.,0
9686,87,1,0.000000,Intel Corporate,0


In [14]:
for entry in sorted(df['Manufacturer'].unique()):
    print(entry)

Amazon Technologies Inc.
Apple, Inc.
AzureWave Technology Inc.
Beijing LT Honway Technology Co.,Ltd
Google, Inc.
Hewlett Packard
Intel Corporate
Murata Manufacturing Co., Ltd.
Raspberry Pi Foundation
Realtek Semiconductor Corp.
Rivet Networks
Samsung Electro-Mechanics(Thailand)
Technicolor CH USA Inc.
Topwell International Holdinds Limited
Ubiquiti Networks Inc.


In [15]:
# This is based on the superset from all records in the database
# the result above is from limiting our dataset to only 10,000
# sessions worth of data

mans = """
Amazon Technologies Inc.
Apple, Inc.
AzureWave Technology Inc.
Beijing LT Honway Technology Co.,Ltd
Google, Inc.
Hewlett Packard
Intel Corporate
Murata Manufacturing Co., Ltd.
Raspberry Pi Foundation
Realtek Semiconductor Corp.
Rivet Networks
Samsung Electro-Mechanics(Thailand)
Technicolor CH USA Inc.
Topwell International Holdinds Limited
Ubiquiti Networks Inc.
""".splitlines()

mapping = {k: v for v, k in enumerate((x for x in mans if len(x)>0),1)}
print(mapping)

{'Amazon Technologies Inc.': 1, 'Apple, Inc.': 2, 'AzureWave Technology Inc.': 3, 'Beijing LT Honway Technology Co.,Ltd': 4, 'Google, Inc.': 5, 'Hewlett Packard': 6, 'Intel Corporate': 7, 'Murata Manufacturing Co., Ltd.': 8, 'Raspberry Pi Foundation': 9, 'Realtek Semiconductor Corp.': 10, 'Rivet Networks': 11, 'Samsung Electro-Mechanics(Thailand)': 12, 'Technicolor CH USA Inc.': 13, 'Topwell International Holdinds Limited': 14, 'Ubiquiti Networks Inc.': 15}


**Categorical data mapping**

For the deep learning algorithm to work, we need to get rid of all categorical data.  For the Manufacturer's we will create a 1:1 mapping of the manufacturer name as per the wireshark OUI lookup dataset and the relative
position of that in our ordered array of unique entries

In [16]:
df['Manufacturer'] = df['Manufacturer'].map(mapping)

In [17]:
df.head()

,SessionPackets,SessionPktLen,SessionPktArrivalTime,Manufacturer,is_iot
0,36384156,126998,-419.030715,15,0
1,9039457,125631,-419.074508,2,0
2,306906857,228847,-419.074864,15,1
3,84313180,83489,-419.072925,15,1
4,18288,127,-434.025740,15,0


In [18]:
df['SessionPackets'] = (df['SessionPackets'] - df['SessionPackets'].mean()) / \
    (df['SessionPackets'].max() - df['SessionPackets'].min())

In [19]:
df['SessionPktLen'] = (df['SessionPktLen'] - df['SessionPktLen'].mean()) / \
    (df['SessionPktLen'].max() - df['SessionPktLen'].min())

In [20]:
df['SessionPktArrivalTime'] = (df['SessionPktArrivalTime'] - df['SessionPktArrivalTime'].mean()) / \
    (df['SessionPktArrivalTime'].max() - df['SessionPktArrivalTime'].min())

In [21]:
df

,SessionPackets,SessionPktLen,SessionPktArrivalTime,Manufacturer,is_iot
0,0.048760,0.254945,-0.103186,15,0
1,0.011819,0.252193,-0.103197,2,0
2,0.414216,0.459968,-0.103197,15,1
3,0.113508,0.167361,-0.103196,15,1
4,-0.000368,-0.000447,-0.106819,15,0
...,...,...,...,...,...
9683,-0.000392,-0.000701,-0.001678,15,0
9684,-0.000390,-0.000693,-0.001629,13,0
9685,-0.000390,-0.000691,-0.001636,15,0
9686,-0.000392,-0.000701,-0.001678,7,0


# Machine learning - session 1

Firstly, we will try to see if we can converge our model, we'll split the dataset as follows:

1. Training set (80%)
2. Test set (20%)
3. We have not set up a validation set as we will continue to validate the model against new captures

The first model will be pretty basic and will only be able to `predict` if session data corresponds to an IoT device or not.

Later runs will refine the model to try to predict the device name from the data that we pass to it.  The DNN will be compriosed of the following layers:

[Input Layer: 4 inputs] -> [Hidden layer 1: 32 nodes] -> [Hidden layer 2: 64 nodes] -> [Hidden layer 3: 32 nodes] -> [Output layer: 1 output]

the `relu` activation function defined as f(x) = max(0, x) will be used for each layer apart from the last layer which will use the sigmoid function.  The adam optimiser will be used and the loss function will be the binary_crossentropy which is best for binary classification problems.


In [22]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)

2.0.0-rc1


In [23]:
features = df.drop('is_iot', axis=1).values

In [ ]:
labels = df['is_iot'].values

In [25]:
labels

array([0, 0, 1, ..., 0, 0, 0])

In [26]:
features

array([[ 4.87599300e-02,  2.54945094e-01, -1.03186141e-01,
         1.50000000e+01],
       [ 1.18193007e-02,  2.52193312e-01, -1.03196750e-01,
         2.00000000e+00],
       [ 4.14215728e-01,  4.59967952e-01, -1.03196836e-01,
         1.50000000e+01],
       ...,
       [-3.89839273e-04, -6.90824970e-04, -1.63587142e-03,
         1.50000000e+01],
       [-3.92207444e-04, -7.00890010e-04, -1.67841770e-03,
         7.00000000e+00],
       [-3.91988594e-04, -7.00890010e-04, -1.67841770e-03,
         1.50000000e+01]])

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [28]:
print(len(labels), len(Y_train), len(Y_test), sep='::')

9688::7750::1938


Again, we will use the following model hyperparameters which are suited for binary classification

 - Output Layer Configuration: `One node with a sigmoid activation unit`.
 - Loss Function: `Cross-Entropy`, also referred to as Logarithmic loss.


In [38]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy

In [39]:
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # Let's create the model
    model = Sequential([
        Dense(32, input_dim=4, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(loss= 'mse' , optimizer= 'adam' , metrics=['accuracy'])
    return model

In [43]:
model = KerasClassifier(build_fn=create_model, verbose=2)

optimizers = ['adam']
inits = ['normal', 'uniform']
epochs = [30, 40]
batches = [5, 6, 8, 10]
param_grid=dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)

# Summarise results
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f'{mean:.4f} ({stdev:.4f}) with: {param}')

Epoch 1/30
 - 1s - loss: 0.1218 - accuracy: 0.8595
Epoch 2/30
 - 1s - loss: 0.1188 - accuracy: 0.8595
Epoch 3/30
 - 1s - loss: 0.1182 - accuracy: 0.8595
Epoch 4/30
 - 1s - loss: 0.1170 - accuracy: 0.8595
Epoch 5/30
 - 1s - loss: 0.1150 - accuracy: 0.8587
Epoch 6/30
 - 1s - loss: 0.1132 - accuracy: 0.8604
Epoch 7/30
 - 1s - loss: 0.1113 - accuracy: 0.8610
Epoch 8/30
 - 1s - loss: 0.1097 - accuracy: 0.8699
Epoch 9/30
 - 1s - loss: 0.1084 - accuracy: 0.8707
Epoch 10/30
 - 1s - loss: 0.1073 - accuracy: 0.8715
Epoch 11/30
 - 1s - loss: 0.1068 - accuracy: 0.8730
Epoch 12/30
 - 1s - loss: 0.1064 - accuracy: 0.8722
Epoch 13/30
 - 1s - loss: 0.1062 - accuracy: 0.8728
Epoch 14/30
 - 1s - loss: 0.1055 - accuracy: 0.8724
Epoch 15/30
 - 1s - loss: 0.1042 - accuracy: 0.8722
Epoch 16/30
 - 1s - loss: 0.1016 - accuracy: 0.8728
Epoch 17/30
 - 1s - loss: 0.0956 - accuracy: 0.8933
Epoch 18/30
 - 1s - loss: 0.0905 - accuracy: 0.8995
Epoch 19/30
 - 1s - loss: 0.0896 - accuracy: 0.8988
Epoch 20/30
 - 1s - l

Epoch 10/30
 - 1s - loss: 0.1160 - accuracy: 0.8560
Epoch 11/30
 - 1s - loss: 0.1149 - accuracy: 0.8624
Epoch 12/30
 - 1s - loss: 0.1141 - accuracy: 0.8630
Epoch 13/30
 - 1s - loss: 0.1133 - accuracy: 0.8636
Epoch 14/30
 - 1s - loss: 0.1131 - accuracy: 0.8632
Epoch 15/30
 - 1s - loss: 0.1130 - accuracy: 0.8638
Epoch 16/30
 - 1s - loss: 0.1126 - accuracy: 0.8638
Epoch 17/30
 - 1s - loss: 0.1126 - accuracy: 0.8639
Epoch 18/30
 - 1s - loss: 0.1125 - accuracy: 0.8638
Epoch 19/30
 - 1s - loss: 0.1123 - accuracy: 0.8638
Epoch 20/30
 - 1s - loss: 0.1118 - accuracy: 0.8638
Epoch 21/30
 - 1s - loss: 0.1106 - accuracy: 0.8639
Epoch 22/30
 - 1s - loss: 0.1093 - accuracy: 0.8639
Epoch 23/30
 - 1s - loss: 0.1061 - accuracy: 0.8667
Epoch 24/30
 - 1s - loss: 0.1016 - accuracy: 0.8883
Epoch 25/30
 - 1s - loss: 0.0971 - accuracy: 0.8930
Epoch 26/30
 - 1s - loss: 0.0949 - accuracy: 0.8934
Epoch 27/30
 - 1s - loss: 0.0931 - accuracy: 0.8937
Epoch 28/30
 - 1s - loss: 0.0930 - accuracy: 0.8936
Epoch 29/30


Epoch 19/40
 - 1s - loss: 0.1019 - accuracy: 0.8726
Epoch 20/40
 - 1s - loss: 0.1000 - accuracy: 0.8699
Epoch 21/40
 - 1s - loss: 0.0968 - accuracy: 0.8821
Epoch 22/40
 - 1s - loss: 0.0957 - accuracy: 0.8945
Epoch 23/40
 - 1s - loss: 0.0940 - accuracy: 0.8991
Epoch 24/40
 - 1s - loss: 0.0934 - accuracy: 0.8980
Epoch 25/40
 - 1s - loss: 0.0929 - accuracy: 0.8988
Epoch 26/40
 - 1s - loss: 0.0921 - accuracy: 0.8990
Epoch 27/40
 - 1s - loss: 0.0918 - accuracy: 0.8991
Epoch 28/40
 - 1s - loss: 0.0918 - accuracy: 0.8991
Epoch 29/40
 - 1s - loss: 0.0910 - accuracy: 0.8984
Epoch 30/40
 - 1s - loss: 0.0899 - accuracy: 0.8995
Epoch 31/40
 - 1s - loss: 0.0900 - accuracy: 0.8993
Epoch 32/40
 - 1s - loss: 0.0911 - accuracy: 0.8964
Epoch 33/40
 - 1s - loss: 0.0890 - accuracy: 0.8990
Epoch 34/40
 - 1s - loss: 0.0892 - accuracy: 0.8991
Epoch 35/40
 - 1s - loss: 0.0885 - accuracy: 0.8991
Epoch 36/40
 - 1s - loss: 0.0891 - accuracy: 0.8980
Epoch 37/40
 - 1s - loss: 0.0895 - accuracy: 0.8966
Epoch 38/40


Epoch 28/30
 - 0s - loss: 0.0900 - accuracy: 0.8936
Epoch 29/30
 - 1s - loss: 0.0896 - accuracy: 0.8945
Epoch 30/30
 - 1s - loss: 0.0931 - accuracy: 0.8914
Epoch 1/30
 - 1s - loss: 0.1327 - accuracy: 0.8427
Epoch 2/30
 - 1s - loss: 0.1249 - accuracy: 0.8512
Epoch 3/30
 - 1s - loss: 0.1237 - accuracy: 0.8514
Epoch 4/30
 - 1s - loss: 0.1225 - accuracy: 0.8525
Epoch 5/30
 - 1s - loss: 0.1217 - accuracy: 0.8550
Epoch 6/30
 - 1s - loss: 0.1211 - accuracy: 0.8554
Epoch 7/30
 - 1s - loss: 0.1205 - accuracy: 0.8548
Epoch 8/30
 - 1s - loss: 0.1200 - accuracy: 0.8556
Epoch 9/30
 - 1s - loss: 0.1197 - accuracy: 0.8556
Epoch 10/30
 - 1s - loss: 0.1190 - accuracy: 0.8552
Epoch 11/30
 - 1s - loss: 0.1185 - accuracy: 0.8556
Epoch 12/30
 - 1s - loss: 0.1178 - accuracy: 0.8552
Epoch 13/30
 - 1s - loss: 0.1169 - accuracy: 0.8556
Epoch 14/30
 - 1s - loss: 0.1164 - accuracy: 0.8554
Epoch 15/30
 - 1s - loss: 0.1155 - accuracy: 0.8581
Epoch 16/30
 - 1s - loss: 0.1147 - accuracy: 0.8630
Epoch 17/30
 - 1s - l

Epoch 37/40
 - 1s - loss: 0.1042 - accuracy: 0.8734
Epoch 38/40
 - 1s - loss: 0.1033 - accuracy: 0.8717
Epoch 39/40
 - 1s - loss: 0.0993 - accuracy: 0.8763
Epoch 40/40
 - 1s - loss: 0.0955 - accuracy: 0.8939
Epoch 1/40
 - 1s - loss: 0.1258 - accuracy: 0.8519
Epoch 2/40
 - 1s - loss: 0.1240 - accuracy: 0.8519
Epoch 3/40
 - 1s - loss: 0.1233 - accuracy: 0.8519
Epoch 4/40
 - 1s - loss: 0.1230 - accuracy: 0.8519
Epoch 5/40
 - 1s - loss: 0.1222 - accuracy: 0.8518
Epoch 6/40
 - 1s - loss: 0.1212 - accuracy: 0.8527
Epoch 7/40
 - 1s - loss: 0.1205 - accuracy: 0.8535
Epoch 8/40
 - 1s - loss: 0.1192 - accuracy: 0.8514
Epoch 9/40
 - 1s - loss: 0.1180 - accuracy: 0.8533
Epoch 10/40
 - 1s - loss: 0.1171 - accuracy: 0.8533
Epoch 11/40
 - 1s - loss: 0.1159 - accuracy: 0.8535
Epoch 12/40
 - 1s - loss: 0.1145 - accuracy: 0.8585
Epoch 13/40
 - 1s - loss: 0.1138 - accuracy: 0.8653
Epoch 14/40
 - 1s - loss: 0.1135 - accuracy: 0.8649
Epoch 15/40
 - 1s - loss: 0.1131 - accuracy: 0.8655
Epoch 16/40
 - 1s - l

...

Epoch 25/30
 - 0s - loss: 0.0934 - accuracy: 0.8974
Epoch 26/30
 - 0s - loss: 0.0911 - accuracy: 0.8993
Epoch 27/30
 - 0s - loss: 0.0893 - accuracy: 0.8999
Epoch 28/30
 - 0s - loss: 0.0883 - accuracy: 0.8993
Epoch 29/30
 - 0s - loss: 0.0875 - accuracy: 0.8993
Epoch 30/30
 - 0s - loss: 0.0871 - accuracy: 0.8986
Epoch 1/30
 - 0s - loss: 0.1258 - accuracy: 0.8519
Epoch 2/30
 - 0s - loss: 0.1240 - accuracy: 0.8519
Epoch 3/30
 - 0s - loss: 0.1236 - accuracy: 0.8519
Epoch 4/30
 - 0s - loss: 0.1235 - accuracy: 0.8519
Epoch 5/30
 - 0s - loss: 0.1227 - accuracy: 0.8519
Epoch 6/30
 - 0s - loss: 0.1212 - accuracy: 0.8525
Epoch 7/30
 - 0s - loss: 0.1210 - accuracy: 0.8531
Epoch 8/30
 - 0s - loss: 0.1194 - accuracy: 0.8545
Epoch 9/30
 - 0s - loss: 0.1178 - accuracy: 0.8533
Epoch 10/30
 - 0s - loss: 0.1163 - accuracy: 0.8541
Epoch 11/30
 - 0s - loss: 0.1149 - accuracy: 0.8589
Epoch 12/30
 - 0s - loss: 0.1135 - accuracy: 0.8649
Epoch 13/30
 - 0s - loss: 0.1130 - accuracy: 0.8645
Epoch 14/30
 - 0s - l

Epoch 14/40
 - 0s - loss: 0.1141 - accuracy: 0.8641
Epoch 15/40
 - 0s - loss: 0.1140 - accuracy: 0.8634
Epoch 16/40
 - 0s - loss: 0.1136 - accuracy: 0.8636
Epoch 17/40
 - 0s - loss: 0.1131 - accuracy: 0.8630
Epoch 18/40
 - 0s - loss: 0.1134 - accuracy: 0.8639
Epoch 19/40
 - 0s - loss: 0.1131 - accuracy: 0.8628
Epoch 20/40
 - 0s - loss: 0.1131 - accuracy: 0.8632
Epoch 21/40
 - 0s - loss: 0.1128 - accuracy: 0.8639
Epoch 22/40
 - 0s - loss: 0.1122 - accuracy: 0.8636
Epoch 23/40
 - 0s - loss: 0.1122 - accuracy: 0.8636
Epoch 24/40
 - 0s - loss: 0.1104 - accuracy: 0.8639
Epoch 25/40
 - 0s - loss: 0.1080 - accuracy: 0.8657
Epoch 26/40
 - 0s - loss: 0.1055 - accuracy: 0.8759
Epoch 27/40
 - 0s - loss: 0.1007 - accuracy: 0.8887
Epoch 28/40
 - 0s - loss: 0.0975 - accuracy: 0.8930
Epoch 29/40
 - 0s - loss: 0.0957 - accuracy: 0.8930
Epoch 30/40
 - 0s - loss: 0.0943 - accuracy: 0.8937
Epoch 31/40
 - 0s - loss: 0.0962 - accuracy: 0.8908
Epoch 32/40
 - 0s - loss: 0.0931 - accuracy: 0.8941
Epoch 33/40


Epoch 13/30
 - 0s - loss: 0.1091 - accuracy: 0.8713
Epoch 14/30
 - 0s - loss: 0.1084 - accuracy: 0.8724
Epoch 15/30
 - 0s - loss: 0.1078 - accuracy: 0.8724
Epoch 16/30
 - 0s - loss: 0.1075 - accuracy: 0.8717
Epoch 17/30
 - 0s - loss: 0.1063 - accuracy: 0.8719
Epoch 18/30
 - 0s - loss: 0.1060 - accuracy: 0.8719
Epoch 19/30
 - 0s - loss: 0.1054 - accuracy: 0.8715
Epoch 20/30
 - 0s - loss: 0.1047 - accuracy: 0.8719
Epoch 21/30
 - 0s - loss: 0.1028 - accuracy: 0.8715
Epoch 22/30
 - 0s - loss: 0.1015 - accuracy: 0.8732
Epoch 23/30
 - 0s - loss: 0.0988 - accuracy: 0.8813
Epoch 24/30
 - 0s - loss: 0.0948 - accuracy: 0.8951
Epoch 25/30
 - 0s - loss: 0.0918 - accuracy: 0.8974
Epoch 26/30
 - 0s - loss: 0.0899 - accuracy: 0.8988
Epoch 27/30
 - 0s - loss: 0.0891 - accuracy: 0.8984
Epoch 28/30
 - 0s - loss: 0.0880 - accuracy: 0.8995
Epoch 29/30
 - 0s - loss: 0.0875 - accuracy: 0.8991
Epoch 30/30
 - 0s - loss: 0.0872 - accuracy: 0.8997
Epoch 1/30
 - 0s - loss: 0.1334 - accuracy: 0.8380
Epoch 2/30
 -

Epoch 22/30
 - 0s - loss: 0.1129 - accuracy: 0.8638
Epoch 23/30
 - 0s - loss: 0.1129 - accuracy: 0.8638
Epoch 24/30
 - 0s - loss: 0.1126 - accuracy: 0.8638
Epoch 25/30
 - 0s - loss: 0.1123 - accuracy: 0.8636
Epoch 26/30
 - 0s - loss: 0.1124 - accuracy: 0.8639
Epoch 27/30
 - 0s - loss: 0.1124 - accuracy: 0.8638
Epoch 28/30
 - 0s - loss: 0.1122 - accuracy: 0.8638
Epoch 29/30
 - 0s - loss: 0.1121 - accuracy: 0.8641
Epoch 30/30
 - 0s - loss: 0.1120 - accuracy: 0.8641
Epoch 1/40
 - 0s - loss: 0.1226 - accuracy: 0.8575
Epoch 2/40
 - 0s - loss: 0.1195 - accuracy: 0.8595
Epoch 3/40
 - 0s - loss: 0.1188 - accuracy: 0.8595
Epoch 4/40
 - 0s - loss: 0.1182 - accuracy: 0.8595
Epoch 5/40
 - 0s - loss: 0.1171 - accuracy: 0.8595
Epoch 6/40
 - 0s - loss: 0.1168 - accuracy: 0.8595
Epoch 7/40
 - 0s - loss: 0.1159 - accuracy: 0.8597
Epoch 8/40
 - 0s - loss: 0.1158 - accuracy: 0.8597
Epoch 9/40
 - 0s - loss: 0.1141 - accuracy: 0.8591
Epoch 10/40
 - 0s - loss: 0.1136 - accuracy: 0.8599
Epoch 11/40
 - 0s - l

Epoch 31/40
 - 0s - loss: 0.1054 - accuracy: 0.8728
Epoch 32/40
 - 0s - loss: 0.1055 - accuracy: 0.8705
Epoch 33/40
 - 0s - loss: 0.1055 - accuracy: 0.8719
Epoch 34/40
 - 0s - loss: 0.1054 - accuracy: 0.8711
Epoch 35/40
 - 0s - loss: 0.1055 - accuracy: 0.8722
Epoch 36/40
 - 0s - loss: 0.1048 - accuracy: 0.8720
Epoch 37/40
 - 0s - loss: 0.1050 - accuracy: 0.8726
Epoch 38/40
 - 0s - loss: 0.1053 - accuracy: 0.8730
Epoch 39/40
 - 0s - loss: 0.1047 - accuracy: 0.8719
Epoch 40/40
 - 0s - loss: 0.1046 - accuracy: 0.8726
Epoch 1/40
 - 0s - loss: 0.1258 - accuracy: 0.8512
Epoch 2/40
 - 0s - loss: 0.1235 - accuracy: 0.8519
Epoch 3/40
 - 0s - loss: 0.1229 - accuracy: 0.8519
Epoch 4/40
 - 0s - loss: 0.1225 - accuracy: 0.8519
Epoch 5/40
 - 0s - loss: 0.1216 - accuracy: 0.8519
Epoch 6/40
 - 0s - loss: 0.1212 - accuracy: 0.8529
Epoch 7/40
 - 0s - loss: 0.1206 - accuracy: 0.8533
Epoch 8/40
 - 0s - loss: 0.1199 - accuracy: 0.8533
Epoch 9/40
 - 0s - loss: 0.1192 - accuracy: 0.8539
Epoch 10/40
 - 0s - l

# Outcome

`Warning: can take a long time to run!!!`

The best results were obtained from the following parameters:


**First run:**

Best: 0.8961290375417279 using {'batch_size': 20, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}

**Second run:**

Best: 0.8961290171300211 using {'batch_size': 5, 'epochs': 30, 'init': 'uniform', 'optimizer': 'adam'}

It is interesting how the optimizers and initialisation algorithm for the neurons changed between the two runs...  The accuracy is good though!!!

**Third run:**

We definitely plateaued here..

<code>
Best: 0.8961290273358745 using {'batch_size': 6, 'epochs': 40, 'init': 'uniform', 'optimizer': 'adam'}
0.8959 (0.0050) with: {'batch_size': 5, 'epochs': 30, 'init': 'normal', 'optimizer': 'adam'}
0.8857 (0.0199) with: {'batch_size': 5, 'epochs': 30, 'init': 'uniform', 'optimizer': 'adam'}
0.8961 (0.0045) with: {'batch_size': 5, 'epochs': 40, 'init': 'normal', 'optimizer': 'adam'}
0.8946 (0.0068) with: {'batch_size': 5, 'epochs': 40, 'init': 'uniform', 'optimizer': 'adam'}
0.8951 (0.0063) with: {'batch_size': 6, 'epochs': 30, 'init': 'normal', 'optimizer': 'adam'}
0.8845 (0.0207) with: {'batch_size': 6, 'epochs': 30, 'init': 'uniform', 'optimizer': 'adam'}
0.8946 (0.0068) with: {'batch_size': 6, 'epochs': 40, 'init': 'normal', 'optimizer': 'adam'}
0.8961 (0.0049) with: {'batch_size': 6, 'epochs': 40, 'init': 'uniform', 'optimizer': 'adam'}
0.8761 (0.0182) with: {'batch_size': 8, 'epochs': 30, 'init': 'normal', 'optimizer': 'adam'}
0.8948 (0.0067) with: {'batch_size': 8, 'epochs': 30, 'init': 'uniform', 'optimizer': 'adam'}
0.8853 (0.0123) with: {'batch_size': 8, 'epochs': 40, 'init': 'normal', 'optimizer': 'adam'}
0.8960 (0.0051) with: {'batch_size': 8, 'epochs': 40, 'init': 'uniform', 'optimizer': 'adam'}
0.8862 (0.0125) with: {'batch_size': 10, 'epochs': 30, 'init': 'normal', 'optimizer': 'adam'}
0.8865 (0.0098) with: {'batch_size': 10, 'epochs': 30, 'init': 'uniform', 'optimizer': 'adam'}
0.8768 (0.0168) with: {'batch_size': 10, 'epochs': 40, 'init': 'normal', 'optimizer': 'adam'}
0.8764 (0.0181) with: {'batch_size': 10, 'epochs': 40, 'init': 'uniform', 'optimizer': 'adam'}
</code>